## Loading dataset 

In [1]:
import os 
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [2]:
! kaggle datasets download -d rahilmehtaucoe2784/fashion

 97% 185M/191M [00:06<00:00, 24.5MB/s]
100% 191M/191M [00:06<00:00, 33.1MB/s]


In [3]:
%%capture
!unzip \*.zip && rm *.zip &> /dev/null

## installing and importing Libraries 

In [4]:
!pip install -q tflite-model-maker
!pip install -q pycocotools

     |████████████████████████████████| 616 kB 4.3 MB/s 
     |████████████████████████████████| 120 kB 91.6 MB/s 
     |████████████████████████████████| 596 kB 81.7 MB/s 
     |████████████████████████████████| 6.4 MB 74.6 MB/s 
     |████████████████████████████████| 1.1 MB 68.7 MB/s 
     |████████████████████████████████| 87 kB 9.2 MB/s 
     |████████████████████████████████| 1.1 MB 55.7 MB/s 
     |████████████████████████████████| 1.2 MB 66.5 MB/s 
     |████████████████████████████████| 234 kB 77.7 MB/s 
     |████████████████████████████████| 77 kB 7.7 MB/s 
     |████████████████████████████████| 3.4 MB 65.9 MB/s 
     |████████████████████████████████| 840 kB 75.1 MB/s 
     |████████████████████████████████| 25.3 MB 1.2 MB/s 
     |████████████████████████████████| 99 kB 12.0 MB/s 
     |████████████████████████████████| 352 kB 79.5 MB/s 
     |████████████████████████████████| 47.7 MB 166 kB/s 
     |████████████████████████████████| 462 kB 95.8 MB/s 
     |██████████████

In [5]:
import numpy as np
import os

#from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)
from tensorflow import keras

## Training 

In [6]:
train_image_dir = '/content/Final Img copy 2/train'
train_annotation_dir = '/content/Final Img copy 2/train anotation'
validate_image_dir = '/content/Final Img copy 2/valid'
validate_anaotation_dir = '/content/Final Img copy 2/valid anotation '

In [7]:

spec = model_spec.get('efficientdet_lite4')

In [8]:
train_dir = object_detector.DataLoader.from_pascal_voc(train_image_dir, train_annotation_dir, label_map={1:"Design Kurta", 2: "Plain kurta", 3: "Casual Shirt", 4:"Formal Shirt", 5: "Desgin sherwani", 6:"Plain sherwani", 7:"tshirt"})

In [9]:
validate_dir = object_detector.DataLoader.from_pascal_voc(validate_image_dir, validate_anaotation_dir, label_map={1:"Design Kurta", 2: "Plain kurta", 3: "Casual Shirt", 4:"Formal Shirt", 5: "Desgin sherwani", 6:"Plain sherwani",7:"tshirt"})

In [11]:
model = object_detector.create(train_dir, 
                               model_spec=spec, 
                               epochs=10, 
                               batch_size=5, 
                               train_whole_model=True, 
                               validation_data=validate_dir
                               )

Epoch 1/10
176/176 [==============================] - 254s 1s/step - det_loss: 1.1445 - cls_loss: 0.9139 - box_loss: 0.0046 - reg_l2_loss: 0.1087 - loss: 1.2533 - learning_rate: 0.0071 - gradient_norm: 2.8211 - val_det_loss: 0.6843 - val_cls_loss: 0.5812 - val_box_loss: 0.0021 - val_reg_l2_loss: 0.1088 - val_loss: 0.7931
Epoch 2/10
176/176 [==============================] - 204s 1s/step - det_loss: 0.5406 - cls_loss: 0.4833 - box_loss: 0.0011 - reg_l2_loss: 0.1089 - loss: 0.6495 - learning_rate: 0.0058 - gradient_norm: 3.3222 - val_det_loss: 0.5904 - val_cls_loss: 0.5691 - val_box_loss: 4.2639e-04 - val_reg_l2_loss: 0.1090 - val_loss: 0.6993
Epoch 3/10
176/176 [==============================] - 205s 1s/step - det_loss: 0.4458 - cls_loss: 0.4058 - box_loss: 8.0098e-04 - reg_l2_loss: 0.1090 - loss: 0.5548 - learning_rate: 0.0051 - gradient_norm: 3.5313 - val_det_loss: 0.4006 - val_cls_loss: 0.3817 - val_box_loss: 3.7653e-04 - val_reg_l2_loss: 0.1090 - val_loss: 0.5096
Epoch 4/10
176/176 

## Evaluating the model 

In [13]:
model.evaluate(validate_dir,batch_size=5)

49/49 [==============================] - 23s 372ms/step



{'AP': 0.6369193,
 'AP50': 0.69317114,
 'AP75': 0.6808171,
 'AP_/Casual Shirt': 0.7512621,
 'AP_/Desgin sherwani': 0.73052377,
 'AP_/Design Kurta': 0.40258652,
 'AP_/Formal Shirt': 0.8168402,
 'AP_/Plain kurta': 0.28033265,
 'AP_/Plain sherwani': 0.5680787,
 'AP_/tshirt': 0.90881133,
 'APl': 0.63695246,
 'APm': -1.0,
 'APs': -1.0,
 'ARl': 0.91306376,
 'ARm': -1.0,
 'ARmax1': 0.8989516,
 'ARmax10': 0.91306376,
 'ARmax100': 0.91306376,
 'ARs': -1.0}

## Saving Model 

In [14]:
model.export(export_dir='/content/')